In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
class Scraper:
    def __init__(self, region, max_comunas, months, years, logs = True, outputs = False):
        import pandas as pd
        import unicodedata as unicode
        import urllib.request as request
        from bs4 import BeautifulSoup
        import sqlite3 as sqlite
        import random
        import os
        import time
        
        self.region = region
        self.max_comunas = max_comunas
        self.months = months
        self.years = years
        self.tipos = ['contrata', 'planta']
        self.codigo_region = int()
        self.logs = logs
        self.outputs = outputs
        self.pd = pd
        self.unicode = unicode
        self.request = request
        self.bs4 = BeautifulSoup
        self.sqlite = sqlite
        self.random = random
        self.os = os
        self.time = time
        self.classifier = self.createClassifier('Michael')
        self.df_region = None
        self.dict_comunas = dict()
        self.planta_contrata_links = dict()
        self.date_remunerations_dict = dict()
        self.file_time = None
        
    # Función que crea el modelo de clasificación de nombres (indica un posible genero de esa persona)
    def createClassifier(self, name_test = 'Ricardo'):  
        import nltk
        from nltk.corpus import names
        
        labeled_names = ([(name, 'H') for name in names.words('male.txt')] + [(name, 'M') for name in names.words('female.txt')]) 
        self.random.shuffle(labeled_names)
        featuresets = [({'last_five_letters':n[-5:]}, gender) for (n, gender) in labeled_names]   
        train_set, test_set = featuresets[100:], featuresets[:100] 
        classifier = nltk.NaiveBayesClassifier.train(train_set) 

        print('### Se ha creado el clasificador de personas ###') if self.outputs else None
        print('TEST with '+ name_test +': ' + str(classifier.classify({'last_five_letters':name_test[-5:]}))) if self.outputs else None
        print('Accuracy of this model: ' + str(nltk.classify.accuracy(classifier, train_set))) if self.outputs else None

        return classifier
    
    # Función que retorna las comunas pertenecientes a una región
    def getComunasFromRegion(self):
        df = self.pd.read_excel('data_regiones/cut_2018_v03.xls')
        df.columns = map(str.lower, df.columns)
        df.columns = df.columns.str.replace(' ', '_')
        df.columns = df.columns.str.split('_').str[:2].str.join(sep='_')
        df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

        if self.region == '':
            df_region = self.pd.DataFrame()
        elif isinstance(self.region, int):
            df_region = df[df['codigo_region'] == self.region]
        elif isinstance(self.region, str):        
            if region.isdigit() == True:
                df_region = df[df['codigo_region'] == int(self.region)]
            else:
                self.region = self.unicode.normalize('NFD', self.region).encode('ascii', 'ignore').decode("utf-8").lower()
                df_region = df[df['nombre_region'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower() == self.region]
        
        if not df_region.empty:            
            self.createDirectories() 
            
        return df_region
    
    # Función que remueve acentos de una cadena de texto
    def removeAccents(self, input_str):
        nfkd_form = self.unicode.normalize('NFKD', input_str)
        return u"".join([c for c in nfkd_form if not self.unicode.combining(c)])
    
    # Creamos los directorios asociados a una region        
    def createDirectories(self):
        path = self.region.lower()

        if self.os.path.exists(path):
            print('\n     El directorio "%s" ya existe.' % path) if self.outputs else None
            self.createLogFile()
        else:
            try:
                self.os.mkdir(path)
            except self.os.OSError:
                print('\n     Creación del directorio "%s" ha fallado.' % path) if self.outputs else None
                self.createLogFile()
                self.appendLogMessage('Creación del directorio "%s" ha fallado.' % path)
            else:
                csv_path = path + '/csv'
                self.os.mkdir(csv_path)
                db_path = path + '/db'
                self.os.mkdir(db_path)
                logs_path = path + '/logs'
                self.os.mkdir(logs_path)
                self.createLogFile()
                
                print('Se ha creado el directorio "%s" con exito.' % path) if self.outputs else None
                self.appendLogMessage('Se ha creado el directorio "%s" con exito.' % path)
    
    
    # Permite crear el archivo .LOG para la ejecución del Scraper actual
    def createLogFile(self):
        import os        
       
        log_file =  self.removeAccents(self.region.lower()) + '/logs/' + self.file_time + '.log'
        
        if not self.os.path.isfile(log_file):
            open(log_file, 'w+')
    
    # Función para agregar mensajes (filas) al archivo .LOG
    def appendLogMessage(self, message = ''):
        from datetime import datetime
        import os

        now = datetime.now()
        date = now.strftime("%d_%m_%Y")
        time = now.strftime('%H:%M:%S.%f')[:-3]
        
        log_file =  self.removeAccents(self.region.lower()) + '/logs/' + self.file_time + '.log'
        
        if message != '':
            full_message = time + ' | ' + message + '\n'
            file = open(log_file, 'a+')
            file.write(full_message)    
    
    # Funfcion que retorna un diccionario con todas las comunas hasta el valor de max_comunas
    def getListComunas(self):
        for i, row in list(self.df_region.iterrows())[:self.max_comunas]:
            self.dict_comunas[row['nombre_comuna']] = [row['codigo_comuna'], row['nombre_region'], row['codigo_region']]
    
    # Funcion para obtener las URLS (primeros enlaces) de las paginas de remuneraciones de trabajdores CONTRATA y PLANTA
    def getPlantaAndContrataURLs(self):
        del_comunas = []
        lista_comunas = self.dict_comunas.keys()
        print('\n     Obteniendo URLs de Planta y Contrata para cada comuna.') if self.outputs else None  
        self.appendLogMessage('Obteniendo URLs de Planta y Contrata para cada comuna.') if self.logs else None
        
        for i, comuna in enumerate(lista_comunas):
            intentos = 0
            print('\n     ' + str(i + 1) + '. Scraping: ' + comuna) if self.outputs else None       
            self.appendLogMessage(str(i + 1) + '. Scraping: ' + comuna) if self.logs else None
            
            while intentos < 3:
                try:               
                    comuna_norm = self.unicode.normalize('NFD', comuna).encode('ascii', 'ignore').decode("utf-8")
                    comuna_norm = comuna_norm.replace(' ', '%20')

                    url = 'https://www.portaltransparencia.cl/PortalPdT/web/guest/home?p_p_id=3&p_p_lifecycle=0&p_p_state=maximized&p_p_mode=view&_3_struts_action=/search/search&_3_redirect=/PortalPdT/web/guest/home?p_auth=gVKA8DxS&p_p_id=3&p_p_lifecycle=1&p_p_state=normal&p_p_state_rcv=1&_3_groupId=0&x=0&y=0&_3_keywords=' + comuna_norm + '&search_term=' + comuna_norm
                    page_1 = self.request.urlopen(url).read().decode('utf-8')
                    self.time.sleep(0.2)
                    
                    soup = self.bs4(page_1)
                    key = 'municipalidad de ' + comuna.lower()
                    tag = soup.find(lambda tag : tag.name == 'a' and key in tag.text.lower())
                    link_1 = tag.get('href')

                    url = link_1
                    page_2 = self.request.urlopen(url).read().decode('utf-8')
                    self.time.sleep(0.2)
                    
                    soup =  self.bs4(page_2)
                    tag = soup.find('div', {'class': 'enlace_ficha_org'}).find('a', {'class': 'estilo_info_ta'})
                    href = tag.get('href')
                    url, sep, cod = href.partition('=')
                    link_2 = url       

                    if cod != '':
                        url = link_2 + '=' + cod
                        page_3 = self.request.urlopen(url).read().decode('utf-8')
                        self.time.sleep(2)
                        
                        soup =  self.bs4(page_3)
                        tag_1 = soup.find('a', attrs={'title': lambda x: x and x.lower() == 'personal a contrata'})
                        tag_2 = soup.find('a', attrs={'title': lambda x: x and x.lower() == 'personal de planta'})
                        self.planta_contrata_links[comuna] = [tag_1.get('href'), tag_2.get('href')]
                        intentos = 3
                        print('     Completado.') if self.outputs else None
                        self.appendLogMessage('Completado.') if self.logs else None

                    else:
                        del_comunas.append(comuna)
                        intentos = 3
                        print('     Se omitira esta comuna, ya que su página no pertenece a Transparencia.') if self.outputs else None
                        self.appendLogMessage('Se omitira esta comuna, ya que su página no pertenece a Transparencia.') if self.logs else None

                except:
                    if intentos < 2:
                        print('     En el intento %s se produjo un error.' % str(intentos + 1)) if self.outputs else None
                        self.appendLogMessage('En el intento %s se produjo un error.' % str(intentos + 1)) if self.logs else None
                        intentos = intentos + 1
                    else:
                        print('     Se excedio el limite de intentos para esta comuna: se omitira.') if self.outputs else None
                        self.appendLogMessage('Se excedio el limite de intentos para esta comuna: se omitira.') if self.logs else None
                        intentos = intentos + 1
                        del_comunas.append(comuna)

        print('\n') if self.outputs else None
        for comuna in del_comunas:
            print('     Se ha eliminado ' + str(comuna) + ' de la lista (no se pudieron rescatar las URLs de Contrata y Planta de esta comuna).') if self.outputs else None
            self.appendLogMessage('Se ha eliminado ' + str(comuna) + ' de la lista (no se pudieron rescatar las URLs de Contrata y Planta de esta comuna).') if self.logs else None
            del self.dict_comunas[comuna]
    
    # Con esta función creamos el un diccionaria individual que almacena 2 DataFrames (1 para la tabla Contrata y otro para la tabla Planta)
    def createPlantaAndContrataDataFrames(self, month, year):
        dataframe_collections = dict()

        for i, comuna in enumerate(list(self.dict_comunas.keys())):
            print('\n     ' + str(i + 1) + '. Scraping: ' + str(comuna)) if self.outputs else None
            self.appendLogMessage(str(i + 1) + '. Scraping: ' + str(comuna)) if self.logs else None
            # getRemuneracionesByDate es la funcion con el algoritmo que hace scraping hasta la tabla final
            dataframe_collections[comuna] = self.getRemuneracionesByDate(month, year, comuna, self.dict_comunas[comuna][0], self.region.lower(), self.dict_comunas[comuna][2], self.planta_contrata_links[comuna])

        return dataframe_collections
    
    # Nos ayuda a definir el genero de una persona utilizando las ultimas 5 letras de su nombre (primer nombre)
    def defineGenderOnDF(self, df):
        df['GENERO'] = 'M'
        genders = []

        for name in df['NOMBRE COMPLETO']:
            name_split = name.split()
            name_split = name_split[2:3]
            first_name = ''

            for i in name_split:
                first_name = first_name + ' ' + str(i)

            genders.append(self.classifier.classify({'last_five_letters':first_name.lower()[-5:]}))

        df['GENERO'] = genders
        return df
    
    # Funcion que se encarga de buscar en las paginas de CONTRATA y PLANTA las tablas de remuneraciones
    # Esta funcion utiliza otra subfuncion llamada recursive() que nos ayuda a explorar las posibles urls hasta llegar a la tabla deseada
    def getRemuneracionesByDate(self, month, year, comuna, codigo_comuna, region, codigo_region, urls = []):
        from urllib.error import URLError, HTTPError     
        
        def recursive(href, i, keys, depth = 3):
            if depth != 0:
                j = 0
                
                while j < 3:
                    try:
                        page = self.request.urlopen(href).read().decode('utf-8')
                        self.time.sleep(1)
                        
                    except HTTPError as e:
                        print('        Ocurrio un error al alcanzar la URL | Error: ' + str(e.code))
                        self.appendLogMessage('Ocurrio un error al alcanzar la URL | Error: ' + str(e.code)) if self.logs else None
                        print('        Reintentando...')
                        self.appendLogMessage('Reintentando...') if self.logs else None
                        j = j + 1

                    else:
                        j = 3
                        soup = self.bs4(page)

                        if soup.find('table'):                
                            tag_ = soup.find('table') 

                            if i == 0:
                                df_contrata = self.pd.read_html(str(tag_))[0]
                                
                                if df_contrata.shape[0] > 1: 
                                    df_contrata.columns = df_contrata.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
                                    df_contrata.columns = map(str.upper, df_contrata.columns)

                                    if 'REMUNERACION BRUTA MENSUALIZADA' in df_contrata.columns:
                                        df_contrata['REMUNERACION BRUTA MENSUALIZADA'] = df_contrata['REMUNERACION BRUTA MENSUALIZADA'].astype(str).str.replace('.', '')
                                    else:
                                        df_contrata['REMUNERACION BRUTA MENSUALIZADA'] = 'NO INFORMA'

                                    if 'REMUNERACION LIQUIDA MENSUALIZADA' in df_contrata.columns:
                                        df_contrata['REMUNERACION LIQUIDA MENSUALIZADA'] = df_contrata['REMUNERACION LIQUIDA MENSUALIZADA'].astype(str).str.replace('.', '')
                                    else:
                                        df_contrata['REMUNERACION LIQUIDA MENSUALIZADA'] = 'NO INFORMA'

                                    df_contrata['TIPO'] = 'CONTRATA'
                                    df_contrata['REGION'] = region
                                    df_contrata['COMUNA'] = comuna
                                    df_contrata['CODIGO COMUNA'] = codigo_comuna
                                    df_contrata['CODIGO REGION'] = codigo_region                            
                                    df_contrata = df_contrata.apply(lambda x: x.astype(str).str.upper())                            
                                    df_contrata = self.defineGenderOnDF(df_contrata)
                                    dfs.append(df_contrata)
                                else:
                                    dfs.append(self.pd.DataFrame())

                            else:
                                df_planta = self.pd.read_html(str(tag_))[0]
                                
                                if df_planta.shape[0] > 1:
                                    df_planta.columns = df_planta.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
                                    df_planta.columns = map(str.upper, df_planta.columns)

                                    if 'REMUNERACION BRUTA MENSUALIZADA' in df_planta.columns:
                                        df_planta['REMUNERACION BRUTA MENSUALIZADA'] = df_planta['REMUNERACION BRUTA MENSUALIZADA'].astype(str).str.replace('.', '')
                                    else:
                                        df_planta['REMUNERACION BRUTA MENSUALIZADA'] = 'NO INFORMA'

                                    if 'REMUNERACION LIQUIDA MENSUALIZADA' in df_planta.columns:
                                        df_planta['REMUNERACION LIQUIDA MENSUALIZADA'] = df_planta['REMUNERACION LIQUIDA MENSUALIZADA'].astype(str).str.replace('.', '')
                                    else:
                                        df_planta['REMUNERACION LIQUIDA MENSUALIZADA'] = 'NO INFORMA'

                                    df_planta['TIPO'] = 'PLANTA'
                                    df_planta['REGION'] = region
                                    df_planta['COMUNA'] = comuna
                                    df_planta['CODIGO COMUNA'] = codigo_comuna
                                    df_planta['CODIGO REGION'] = codigo_region
                                    df_planta = df_planta.apply(lambda x: x.astype(str).str.upper())
                                    df_planta = self.defineGenderOnDF(df_planta)
                                    dfs.append(df_planta)
                                else:
                                    dfs.append(self.pd.DataFrame())

                        else:
                            for key in keys:               
                                if soup.find(lambda tag : tag.name == 'a' and key.lower() in tag.text.lower()):                        
                                    tag = soup.find(lambda tag : tag.name == 'a' and key.lower() in tag.text.lower())
                                    url = tag.get('href')
                                    keys.remove(key)                                
                                    recursive(url, i, keys, depth - 1)


        dfs = list()
        encontrado = False                

        for i, url in enumerate(urls):
            print('        Buscando en: ' + url) if self.outputs else None 
            self.appendLogMessage('Buscando en: ' + url) if self.logs else None
            keys = [year, month, 'municipal'] # Esta lista con posibles palabras nos permite discriminar entre las urls de la pagína de transparencia, se elimina un llave de la lista cada vez que haya un match 
            j = 0

            while j < 3:
                try:
                    page = self.request.urlopen(url).read().decode('latin-1')
                    self.time.sleep(1)

                except HTTPError as e:
                    print('        Ocurrio un error al alcanzar la URL inicial | Error code: ' + str(e.code)) if self.outputs else None
                    self.appendLogMessage('Ocurrio un error al alcanzar la URL inicial | Error code: ' + str(e.code)) if self.logs else None
                    print('        Reintentando...') if self.outputs else None
                    self.appendLogMessage('Reintentando...') if self.logs else None
                    j = j + 1

                else:
                    j = 3
                    soup =  self.bs4(page)

                    for key in keys:
                        if soup.find(lambda tag : tag.name == 'a' and key in tag.text.lower()): 
                            tag = soup.find(lambda tag : tag.name == 'a' and key in tag.text.lower())
                            href = tag.get('href')

                            if url != None:
                                break

                    recursive(href, i, keys, 5)              

            if len(dfs) <= i:
                print('        El formato no coincide: puede que no hayan registros para esta fecha, la tabla se encuentra en formato PDF u ocurrio un error HTTP.') if self.outputs else None 
                self.appendLogMessage('El formato no coincide: puede que no hayan registros para esta fecha, la tabla se encuentra en formato PDF u ocurrio un error HTTP.') if self.logs else None
                dfs.append(self.pd.DataFrame())
            else:
                print('        Resultado: Se ha encontrado una tabla y se han rescatado los datos.') if self.outputs else None
                self.appendLogMessage('Resultado: Se ha encontrado una tabla y se han rescatado los datos.') if self.logs else None

        return dfs
    
    # Con esta función podemos exportar cada uno de los DF dentro del Diccionario de DataFrames a formato .CSV
    def exportToCSV(self, region, comuna, dataframes_collection, year, month):          
        try:
            for i, df in enumerate(dataframes_collection):
                t = 'contrata' if i == 0 else 'planta'

                if (df.empty != True) and (df.shape[0] > 1):
                    df.to_csv(self.removeAccents(region.lower()) + '/csv/' + self.removeAccents(comuna.lower()) + '_' + str(year) + '_' + month.lower() + '_' + t + '.csv')
                    print('     La tabla "' + t + '" fue exportada exitosamente.') if self.outputs else None
                    self.appendLogMessage('La tabla "' + t + '" fue exportada exitosamente.') if self.logs else None
                else:
                    print('     La tabla "' + t + '" esta vacia, no se pudieron exportar los datos de esta comuna.') if self.outputs else None
                    self.appendLogMessage('La tabla "' + t + '" esta vacia, no se pudieron exportar los datos de esta comuna.') if self.logs else None
        except:
            print('     Ocurrio un error al exportar los datos de esta comuna.') if self.outputs else None
            self.appendLogMessage('Ocurrio un error al exportar los datos de esta comuna.') if self.logs else None
    
    
    '''
        De aquí en adelante se definen funciones para trabajar con la BD creada o que crearemos en
        SQLite, tales como una función para conectarnos, crear una base de datos para la región dada
        y consultas especificas: para el alcance de este proyecto solo se definieron funciones de inserción
        de datos.
    '''
    def sqlConnection(self):
        try:
            conn = self.sqlite.connect(self.region.lower() + '/db/' + self.region.lower() + '.db')
            return conn

        except self.sqlite.Error as e:
            print('\n           Ha ocurrido el siguiente ERROR: %s, al intentar conectarse a la DB.' %e) if self.outputs else None
            self.appendLogMessage('Ha ocurrido el siguiente ERROR: %s, al intentar conectarse a la DB.' %e) if self.logs else None
    
    def createDB(self):
        try:
            conn = self.sqlConnection()
            cursorObj = conn.cursor()
            query_1 = '''
                    CREATE TABLE IF NOT EXISTS region(
                        id_region INTEGER PRIMARY KEY AUTOINCREMENT,
                        codigo_region INTEGER NOT NULL,
                        nombre_region VARCHAR(30) NOT NULL
                        );
                    '''

            query_2 = '''
                    CREATE TABLE IF NOT EXISTS comuna(
                        id_comuna INTEGER PRIMARY KEY AUTOINCREMENT,
                        codigo_region INTEGER NOT NULL,
                        codigo_comuna INTEGER NOT NULL,
                        nombre_comuna VARCHAR(30) NOT NULL
                        );
                    '''

            query_3 = '''
                    CREATE TABLE IF NOT EXISTS persona(
                        id_persona INTEGER PRIMARY KEY AUTOINCREMENT,
                        tipo_persona VARCHAR(30) NOT NULL,
                        nombre_persona VARCHAR(100) NOT NULL,
                        genero_persona CHAR NOT NULL,
                        formacion_persona VARCHAR(100) NOT NULL
                        );
                    '''

            query_4 = '''
                    CREATE TABLE IF NOT EXISTS remuneracion(
                        id_remuneracion INTEGER PRIMARY KEY AUTOINCREMENT,
                        id_persona INTEGER NOT NULL,
                        codigo_comuna INTEGER NOT NULL,
                        anio_remuneracion INTEGER NOT NULL,
                        mes_remuneracion VARCHAR(15) NOT NULL,
                        estamento_remuneracion VARCHAR(100) NOT NULL,
                        cargo_remuneracion VARCHAR(100) NOT NULL,
                        grado_remuneracion INT NOT NULL,
                        asignaciones_remuneracion VARCHAR(50) NOT NULL,
                        unidad_monetaria_remuneracion VARCHAR(20) NOT NULL,
                        cantidad_bruta_remuneracion VARCHAR(20),
                        cantidad_liquida_remuneracion VARCHAR(20),
                        horas_extras_diurnas_remuneracion VARCHAR(20) NOT NULL,
                        horas_extras_nocturnas_remuneracion VARCHAR(20) NOT NULL,
                        horas_extras_festivos_remuneracion VARCHAR(20) NOT NULL,
                        fecha_inicio_remuneracion VARCHAR(20) NOT NULL,
                        fecha_termino_remuneracion VARCHAR(20) NOT NULL,
                        declaracion_patrimonios_remuneracion VARCHAR(100),
                        viaticos_remuneracion VARCHAR(100),
                        observaciones_remuneracion VARCHAR(200)
                        );
                    '''
            sqls = [query_1, query_2, query_3, query_4]

            for sql in sqls:
                cursorObj.execute(sql)
                conn.commit()

            print('\n           Base de Datos creada con exito.') if self.outputs else None
            self.appendLogMessage('Base de Datos creada con exito.') if self.logs else None
            conn.close()

        except self.sqlite.Error as e:
            print('\n           Ha ocurrido el siguiente ERROR: %s, al intentar crear la DB.' %e) if self.outputs else None
            self.appendLogMessage('Ha ocurrido el siguiente ERROR: %s, al intentar crear la DB.' %e) if self.logs else None
                  
    def insertRegion(self):
        try:
            conn = self.sqlConnection()
            cursorObj = conn.cursor()       
            cursorObj.execute('SELECT id_region FROM region WHERE codigo_region = ? LIMIT 1', [self.codigo_region])
            id_region = cursorObj.fetchone()
            conn.close()        

            if id_region == None: #Verificamos se encuentra o no en nuestra BD, si no existe entonces la Insertamos
                conn = self.sqlConnection()
                cursorObj = conn.cursor()
                cursorObj.execute('''
                    INSERT INTO region
                    (codigo_region, nombre_region) 
                    VALUES(?, ?)
                ''', [self.codigo_region, self.region.lower()])
                conn.commit()
                conn.close()

            print('\n           Región insertada con exito.') if self.outputs else None
            self.appendLogMessage('Región insertada con exito.') if self.logs else None
            conn.close()

        except self.sqlite.Error as e:
            print('\n           Ha ocurrido el siguiente ERROR: %s, al intentar insertar la Región.' %e) if self.outputs else None 
            self.appendLogMessage('Ha ocurrido el siguiente ERROR: %s, al intentar insertar la Región.' %e) if self.logs else None
            
    def insertComuna(self, comuna):
        try:
            conn = self.sqlConnection()
            cursorObj = conn.cursor()       
            cursorObj.execute('SELECT id_comuna FROM comuna WHERE codigo_comuna = ? LIMIT 1', [comuna[1]])
            id_comuna = cursorObj.fetchone()
            conn.close()        
            
            if id_comuna == None: #Verificamos se encuentra o no en nuestra BD, si no existe entonces la Insertamos
                conn = self.sqlConnection()
                cursorObj = conn.cursor()
                cursorObj.execute('''
                    INSERT INTO comuna
                    (codigo_region, codigo_comuna, nombre_comuna) 
                    VALUES(?, ?, ?)
                ''', comuna)
                conn.commit()
                conn.close()
                              
            conn.close()

        except self.sqlite.Error as e:
            print('\n           Ha ocurrido el siguiente ERROR: %s, al intentar insertar los datos de esta Comuna.' %e) if self.outputs else None
            self.appendLogMessage('Ha ocurrido el siguiente ERROR: %s, al intentar insertar los datos de esta Comuna.' %e) if self.logs else None
    
    def insertPersona(self, persona):
        try:
            conn = self.sqlConnection()
            cursorObj = conn.cursor()       
            cursorObj.execute('SELECT id_persona FROM persona WHERE nombre_persona = ? LIMIT 1', [persona[1]])
            id_persona = cursorObj.fetchone()
            conn.close()        

            if id_persona == None: #Verificamos se encuentra o no en nuestra BD, si no existe entonces la Insertamos
                conn = self.sqlConnection()
                cursorObj = conn.cursor()
                cursorObj.execute('''
                    INSERT INTO persona
                    (tipo_persona, nombre_persona, genero_persona, formacion_persona) 
                    VALUES(?, ?, ?, ?)
                ''', persona)
                conn.commit()
                conn.close()
            
            else:
                print('           El usuario ' + str(persona[1].upper()) + ' ya existe en la DB. No fue necesario insertarlo nuevamente.') if self.outputs else None
                self.appendLogMessage('El usuario ' + str(persona[1].upper()) + ' ya existe en la DB. No fue necesario insertarlo nuevamente.') if self.logs else None
            
            conn.close()

        except self.sqlite.Error as e:
            print('\n           Ha ocurrido el siguiente ERROR: %s, al intentar insertar los datos de esta Persona.' %e) if self.outputs else None 
            self.appendLogMessage('Ha ocurrido el siguiente ERROR: %s, al intentar insertar los datos de esta Persona.' %e) if self.logs else None
    
    def insertRemuneracion(self, remuneracion):
        try:
            conn = self.sqlConnection()
            cursorObj = conn.cursor()       
            cursorObj.execute('SELECT id_persona FROM persona WHERE nombre_persona = ? LIMIT 1', [remuneracion[0]])
            id_persona = cursorObj.fetchone()
            conn.close()

            if id_persona != None: #Verificamos se encuentra o no en nuestra BD, si no existe entonces la Insertamos
                conn = self.sqlConnection()
                cursorObj = conn.cursor()
                cursorObj.execute('SELECT id_remuneracion FROM remuneracion WHERE id_persona = ? AND codigo_comuna = ? AND fecha_inicio_remuneracion = ? AND fecha_termino_remuneracion = ? LIMIT 1', [id_persona[0], remuneracion[1], remuneracion[14], remuneracion[15]])
                id_remuneracion = cursorObj.fetchone()
                conn.close()

                if id_remuneracion == None:
                    remuneracion[0] = id_persona[0]
                    conn = self.sqlConnection()
                    cursorObj = conn.cursor()
                    cursorObj.execute('''
                        INSERT INTO remuneracion
                            (id_persona, codigo_comuna, anio_remuneracion, mes_remuneracion,
                            estamento_remuneracion, cargo_remuneracion, grado_remuneracion,
                            asignaciones_remuneracion, unidad_monetaria_remuneracion, cantidad_bruta_remuneracion,
                            cantidad_liquida_remuneracion, horas_extras_diurnas_remuneracion,
                            horas_extras_nocturnas_remuneracion, horas_extras_festivos_remuneracion,
                            fecha_inicio_remuneracion, fecha_termino_remuneracion, declaracion_patrimonios_remuneracion ,
                            viaticos_remuneracion, observaciones_remuneracion) 
                        VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', remuneracion)
                    conn.commit()
                    conn.close()
                
                else:
                    conn = self.sqlConnection()
                    cursorObj = conn.cursor()       
                    cursorObj.execute('SELECT nombre_persona FROM persona WHERE id_persona = ? LIMIT 1', [id_persona[0]])
                    nombre_persona = cursorObj.fetchone()
                    conn.close() 
                    print('           La remuneración con código ' + str(id_remuneracion[0]) + ' ya existe en la DB. Correspone al usuario ' + str(nombre_persona[0]).upper() + ' y no fue necesario insertarla nuevamente.') if self.outputs else None
                    self.appendLogMessage('La remuneración con código ' + str(id_remuneracion[0]) + ' ya existe en la DB. Correspone al usuario ' + str(nombre_persona[0]).upper() + ' y no fue necesario insertarla nuevamente.') if self.logs else None

        except self.sqlite.Error as e:
            print('\n           Ha ocurrido el siguiente ERROR: %s, al intentar insertar los datos de esta Remuneracion.' %e) if self.outputs else None
            self.appendLogMessage('Ha ocurrido el siguiente ERROR: %s, al intentar insertar los datos de esta Remuneracion.' %e) if self.logs else None
    
    
    # Esta función es una macro función que nos ayudara a recorrer nuestro Diccionario de DataFrame (siguiendo el patron año -> mes -> comuna) e insertarlo en la DB
    def insertRemuneracionesOnDB(self, from_csv = True):
        import numpy as np

        print('\n     # COMENZANDO INSERCIÓN DE DATOS') if self.outputs else None
        print('     ---------------------------------------------') if self.outputs else None
        
        for year in self.years:
            for month in self.months:
                print('\n        ' + year + '/ ' + month) if self.outputs else None
                for i, comuna in enumerate(self.dict_comunas.keys()):
                    print('\n        ' + str(i + 1) + '. ' + comuna) if self.outputs else None
                    for j, t in enumerate(self.tipos):                    
                        print('\n           Insertando datos de ' + t.upper() + '...\n') if self.outputs else None
                        
                        if from_csv: 
                            if self.os.path.exists(self.removeAccents(self.region.lower()) + '/csv/' + self.removeAccents(comuna.lower()) + '_' + year + '_' + month.lower() + '_' + t + '.csv'):
                                df = self.pd.read_csv(self.region.lower() + '/csv/' + comuna.lower() + '_' + year + '_' + month.lower() + '_' + t + '.csv')
                                cols = df.select_dtypes(include=[np.object]).columns
                                df[cols] = df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

                                for i, row in df.iterrows():
                                    try:
                                        persona = [row['TIPO'], row['NOMBRE COMPLETO'], row['GENERO'], row['CALIFICACION PROFESIONAL O FORMACION']]
                                        comuna_ = [row['CODIGO REGION'], row['CODIGO COMUNA'], row['COMUNA']]
                                        remuneracion = [
                                            row['NOMBRE COMPLETO'], row['CODIGO COMUNA'], row['ANO'], row['MES'], row['ESTAMENTO'], 
                                            row['CARGO O FUNCION'], row['GRADO EUS / CARGO CON JORNADA'], row['ASIGNACIONES ESPECIALES'], 
                                            row['UNIDAD MONETARIA'], row['REMUNERACION BRUTA MENSUALIZADA'], row['REMUNERACION LIQUIDA MENSUALIZADA'], 
                                            row['MONTOS Y HORAS EXTRAORDINARIAS DIURNAS'], row['MONTOS Y HORAS EXTRAORDINARIAS NOCTURNAS'],
                                            row['MONTOS Y HORAS EXTRAORDINARIAS FESTIVAS'], row['FECHA DE INICIO DD/MM/AA'], row['FECHA DE TERMINO DD/MM/AA'],
                                            row['DECLARACION DE INTERESES Y PATRIMONIO (PUBLICACION POR BUENA PRACTICA)'], row['VIATICOS'],
                                            row['OBSERVACIONES']
                                        ]
                                        
                                        self.insertComuna(comuna_)
                                        self.insertPersona(persona)                                        
                                        self.insertRemuneracion(remuneracion)
                                        print('           Inserción de datos terminado.') if self.outputs else None

                                    except:
                                        print('           Ocurrio un error al intentar insertar los datos de este registro: ' + str(year) + '/' + str(month) + '/' + str(comuna) + '/' + str(t)) if self.outputs else None
                            else:
                                print('           No se han encontrado registros/archivos CSV para esta comuna en la fecha dada: ' + str(year) + '/' + str(month) + '/' + str(comuna) + '/' + str(t)) if self.outputs else None
                        else:
                            df = self.date_remunerations_dict[year][month][comuna][j]
                            
                            if not df.empty:
                                for i, row in df.iterrows():
                                    try:
                                        persona = [row['TIPO'], row['NOMBRE COMPLETO'], row['GENERO'], row['CALIFICACION PROFESIONAL O FORMACION']]
                                        comuna_ = [row['CODIGO REGION'], row['CODIGO COMUNA'], row['COMUNA']]
                                        remuneracion = [
                                            row['NOMBRE COMPLETO'], row['CODIGO COMUNA'], row['ANO'], row['MES'], row['ESTAMENTO'], 
                                            row['CARGO O FUNCION'], row['GRADO EUS / CARGO CON JORNADA'], row['ASIGNACIONES ESPECIALES'], 
                                            row['UNIDAD MONETARIA'], row['REMUNERACION BRUTA MENSUALIZADA'], row['REMUNERACION LIQUIDA MENSUALIZADA'], 
                                            row['MONTOS Y HORAS EXTRAORDINARIAS DIURNAS'], row['MONTOS Y HORAS EXTRAORDINARIAS NOCTURNAS'],
                                            row['MONTOS Y HORAS EXTRAORDINARIAS FESTIVAS'], row['FECHA DE INICIO DD/MM/AA'], row['FECHA DE TERMINO DD/MM/AA'],
                                            row['DECLARACION DE INTERESES Y PATRIMONIO (PUBLICACION POR BUENA PRACTICA)'], row['VIATICOS'],
                                            row['OBSERVACIONES']
                                            ]

                                        self.insertComuna(comuna_)
                                        self.insertPersona(persona)                                    
                                        self.insertRemuneracion(remuneracion)
                                        print('           Inserción de datos terminado.') if self.outputs else None

                                    except:
                                        print('           Ocurrio un error al intentar insertar los datos de este registro: ' + str(year) + '/' + str(month) + '/' + str(comuna) + '/' + str(t)) if self.outputs else None
                            else:
                                print('           No se han encontrado un DF CSV para esta comuna en las fechas dadas dentro del Diccionario de DataFrames: ' + str(year) + '/' +  str(month) + '/' + str(comuna) + '/' + str(t)) if self.outputs else None
                        
    
    # Esta es la función principal cuyo proposito es el de automatizar todo el proceso de Web Scraping, la cual esta dividida en etapas que siguen una logica de: validacion, extracción, transformación y carga.
    def run(self):
        from IPython.display import display, HTML
        from datetime import datetime
        import time      
        import locale
        
        self.file_time = datetime.now().strftime("%d_%m_%Y") + '_' + datetime.now().strftime("%H%M")        
        locale.setlocale(locale.LC_ALL, 'spanish')
        start_time = time.time()        
        start_running = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        
        print('### INICIANDO SCRAPING ###')
        print('\n# PARTE 0: Verificando datos iniciales.') if self.outputs else None   
        print('---------------------------------------------') if self.outputs else None   
        
        self.region = self.removeAccents(self.region)
        self.df_region = self.getComunasFromRegion()
        
        self.appendLogMessage('El Código se comenzo a ejecutar a las: ' + start_running) if self.logs else None
        self.appendLogMessage('### INICIANDO SCRAPING ###') if self.logs else None
        self.appendLogMessage('# PARTE 0: Verificando datos iniciales.') if self.logs else None
        self.appendLogMessage('---------------------------------------------') if self.logs else None
        
        if self.df_region.empty:
            print('\n     ERROR: No se ha encontrado ninguna región con el nombre de: ' + self.region + '.') if self.outputs else None
            self.appendLogMessage('ERROR: No se ha encontrado ninguna región con el nombre de: ' + self.region + '.') if self.logs else None
            return None
        
        else:
            print('\n     Se han encontrado las siguientes comunas para la región de: ' + self.region + '.') if self.outputs else None
            self.appendLogMessage('Se han encontrado las siguientes comunas para la región de: ' + self.region + '.') if self.logs else None
            display(HTML(self.df_region.to_html())) if self.outputs else None 
        
        if len(self.months) == 0 or len(self.years) == 0:
            import datetime
            import locale
            locale.setlocale(locale.LC_ALL, 'spanish')

            if len(self.years) == 0:
                now = datetime.datetime.now()
                self.years = [str(now.year)]

            if len(months) == 1:            
                now = datetime.datetime.now()
                self.months = [str(now.strftime('%B')).capitalize()]

            print('\n     Algunos argumentos no fueron dados, se realizara el scraping con la fecha/s: ' + str(self.months) + str(self.years)) if self.outputs else None
            self.appendLogMessage('Algunos argumentos no fueron dados, se realizara el scraping con la fecha/s: ' + str(self.months) + str(self.years)) if self.logs else None
        
        self.getListComunas()
        self.codigo_region = int(list(self.dict_comunas.values())[0][2])
        print('\n     Las ' + str(self.max_comunas) + ' comunas para realizar Scraping son: ' + str(list(self.dict_comunas.keys())) + ', siendo "' + str(list(self.dict_comunas.keys())[0]) + '" la capital regional.')
        self.appendLogMessage('Las ' + str(self.max_comunas) + ' comunas para realizar Scraping son: ' + str(list(self.dict_comunas.keys())) + ', siendo "' + str(list(self.dict_comunas.keys())[0]) + '" la capital regional.') if self.logs else None
        self.getPlantaAndContrataURLs()
        print('\n     La lista de URLs obtenidas es: ' + str(self.planta_contrata_links)) if self.outputs else None
        self.appendLogMessage('La lista de URLs obtenidas es: ' + str(self.planta_contrata_links)) if self.logs else None
        
        print('\n## PARTE 1: Creando DataFrames por fechas.') if self.outputs else None
        self.appendLogMessage('## PARTE 1: Creando DataFrames por fechas.') if self.logs else None
        print('---------------------------------------------') if self.outputs else None
        self.appendLogMessage('---------------------------------------------') if self.logs else None
        
        for year in self.years:
            self.date_remunerations_dict[year] = dict()
            for month in self.months:
                self.date_remunerations_dict[year][month] = dict()
                print('\n     ' + str(year) + '/' + month) if self.outputs else None
                self.appendLogMessage(str(year) + '/' + month) if self.logs else None
                
                self.date_remunerations_dict[year][month] = self.createPlantaAndContrataDataFrames(month, year)  
        
        print('\n### PARTE 2: Exportando datos a CSV.') if self.outputs else None
        self.appendLogMessage('### PARTE 2: Exportando datos a CSV.') if self.logs else None
        print('---------------------------------------------') if self.outputs else None
        self.appendLogMessage('---------------------------------------------') if self.logs else None
        
        for year in self.years:
            for month in self.months:
                print('\n     ' + str(year) + '/' + month) if self.outputs else None
                self.appendLogMessage(str(year) + '/' + month) if self.logs else None
                
                for i, comuna in enumerate(self.dict_comunas.keys()):
                    print('\n' + str(i + 1) + '. ' + str(comuna)) if self.outputs else None
                    self.appendLogMessage(str(i + 1) + '. ' + str(comuna)) if self.logs else None
                    
                    self.exportToCSV(self.region, comuna, self.date_remunerations_dict[year][month][comuna], year, month)
                    
        print('\n#### PARTE 3: Creando Base de Datos') if self.outputs else None
        self.appendLogMessage('#### PARTE 3: Creando Base de Datos') if self.logs else None
        print('---------------------------------------------') if self.outputs else None
        self.appendLogMessage('---------------------------------------------') if self.logs else None
        self.createDB()
        self.insertRegion()
        self.insertRemuneracionesOnDB(False)
        
        print('\n! --- WEB SCRAPING FINALIZADO EN %s SEGUNDOS --- !' % (time.time() - start_time))
        self.appendLogMessage('! --- WEB SCRAPING FINALIZADO EN %s SEGUNDOS --- !' % (time.time() - start_time)) if self.logs else None

In [3]:
### El algoritmo tiene dependencias con: Pandas, Unicodedata, URLlib, BeautifoulSoup 4, SQLite y NLTK.
### Es necesario descargar estas librearias y tenerlas disponibles en el entorno de trabajo.
### Ademas, es necesario descargar NLTK luego de haberlo instalado: nltk.download()
### La clase SCRAPER necesita ser instanciada con los siguientes argumentos: cadena de texto con el nombre de la región, el número máximo de comunas
### una lista de meses y otra de años de donde se desea extraer las remuneraciones.
### Ademas es posible pasarle un valor booleano para activar la creación de un archivo .LOG del proceso, asi como también otro valor booleano si se desea 
### hacer un print en pantalla de todos los mensajes del proceso.

region = 'maÚle'
months = ['Enero', 'Febrero', 'Marzo']
years = ['2019']
max_comunas = 7

web_scraper = Scraper(region, max_comunas, months, years, True, True)

### Se ha creado el clasificador de personas ###
TEST with Michael: H
Accuracy of this model: 0.9416114227434982


In [4]:
web_scraper.run()

### INICIANDO SCRAPING ###

# PARTE 0: Verificando datos iniciales.
---------------------------------------------
Se ha creado el directorio "maule" con exito.

     Se han encontrado las siguientes comunas para la región de: maule.


,codigo_region,nombre_region,codigo_provincia,nombre_provincia,codigo_comuna,nombre_comuna
111,7,Maule,71,Talca,7101,Talca
112,7,Maule,71,Talca,7102,Constitución
113,7,Maule,71,Talca,7103,Curepto
114,7,Maule,71,Talca,7104,Empedrado
115,7,Maule,71,Talca,7105,Maule
116,7,Maule,71,Talca,7106,Pelarco
117,7,Maule,71,Talca,7107,Pencahue
118,7,Maule,71,Talca,7108,Río Claro
119,7,Maule,71,Talca,7109,San Clemente
120,7,Maule,71,Talca,7110,San Rafael



     Las 7 comunas para realizar Scraping son: ['Talca', 'Constitución', 'Curepto', 'Empedrado', 'Maule', 'Pelarco', 'Pencahue'], siendo "Talca" la capital regional.

     Obteniendo URLs de Planta y Contrata para cada comuna.

     1. Scraping: Talca
     Completado.

     2. Scraping: Constitución
     Completado.

     3. Scraping: Curepto
     Se omitira esta comuna, ya que su página no pertenece a Transparencia.

     4. Scraping: Empedrado
     Completado.

     5. Scraping: Maule
     Completado.

     6. Scraping: Pelarco
     Completado.

     7. Scraping: Pencahue
     Completado.


     Se ha eliminado Curepto de la lista (no se pudieron rescatar las URLs de Contrata y Planta de esta comuna).

     La lista de URLs obtenidas es: {'Talca': ['https://www.portaltransparencia.cl/PortalPdT/pdtta/-/ta/MU312/PR/PCONT', 'https://www.portaltransparencia.cl/PortalPdT/pdtta/-/ta/MU312/PR/PPLANT'], 'Constitución': ['https://www.portaltransparencia.cl/PortalPdT/pdtta/-/ta/MU064/PR/PCONT

        Resultado: Se ha encontrado una tabla y se han rescatado los datos.

     5. Scraping: Pelarco
        Buscando en: https://www.portaltransparencia.cl/PortalPdT/pdtta/-/ta/MU206/PR/PCONT
        El formato no coincide: puede que no hayan registros para esta fecha, la tabla se encuentra en formato PDF u ocurrio un error HTTP.
        Buscando en: https://www.portaltransparencia.cl/PortalPdT/pdtta/-/ta/MU206/PR/PPLANT
        El formato no coincide: puede que no hayan registros para esta fecha, la tabla se encuentra en formato PDF u ocurrio un error HTTP.

     6. Scraping: Pencahue
        Buscando en: https://www.portaltransparencia.cl/PortalPdT/pdtta/-/ta/MU209/PR/PCONT
        Resultado: Se ha encontrado una tabla y se han rescatado los datos.
        Buscando en: https://www.portaltransparencia.cl/PortalPdT/pdtta/-/ta/MU209/PR/PPLANT
        Resultado: Se ha encontrado una tabla y se han rescatado los datos.

### PARTE 2: Exportando datos a CSV.
-----------------------------

           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inserción de datos terminado.
           Inser

...

           Inserción de datos terminado.
           El usuario BECERRA GAJARDO, BLAS ENRIQUE ya existe en la DB. No fue necesario insertarlo nuevamente.
           La remuneración con código 13 ya existe en la DB. Correspone al usuario BECERRA GAJARDO, BLAS ENRIQUE y no fue necesario insertarla nuevamente.
           Inserción de datos terminado.
           El usuario BERNAL BARRIOS, JONATHAN ALEJANDRO ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario BRAVO ALVAREZ, JUAN PATRICIO ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario BRAVO HENRIQUEZ, JAVIER ANDRES ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario BRAVO JAUREGUI, JENNIFER CRISTINA ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario BRAVO MO

           El usuario GARRIDO PARRA, VANESSA DANIELA ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario GATICA ROCHA, IGNACIO DEL CARMEN ya existe en la DB. No fue necesario insertarlo nuevamente.
           La remuneración con código 51 ya existe en la DB. Correspone al usuario GATICA ROCHA, IGNACIO DEL CARMEN y no fue necesario insertarla nuevamente.
           Inserción de datos terminado.
           El usuario GERLI RODRIGUEZ, GIULIANO OSEAS ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario GONZALEZ CARIZ, JUAN ENRIQUE ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario GONZALEZ CARREÑO, MARCELA PAOLA ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario GONZALEZ ESPINOZA, KATHERINE XIMENA ya existe en

           El usuario CARRERA SAAVEDRA, PILAR ANDREA ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario CARTES AVENDAÑO, SANDRA VALERIA ya existe en la DB. No fue necesario insertarlo nuevamente.
           La remuneración con código 143 ya existe en la DB. Correspone al usuario CARTES AVENDAÑO, SANDRA VALERIA y no fue necesario insertarla nuevamente.
           Inserción de datos terminado.
           El usuario CASTILLO AGUILAR, REGINA VILMA EVA ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario CASTILLO CASTILLO, NIRSON IGOR ya existe en la DB. No fue necesario insertarlo nuevamente.
           La remuneración con código 145 ya existe en la DB. Correspone al usuario CASTILLO CASTILLO, NIRSON IGOR y no fue necesario insertarla nuevamente.
           Inserción de datos terminado.
           El usuario CASTILLO TAPIA, MARTA IVON ya existe en la

           El usuario HERNANDEZ DURAN, PATRICIA EUGENIA ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario HERNANDEZ FUENTES, RUBEN SEBASTIAN ya existe en la DB. No fue necesario insertarlo nuevamente.
           La remuneración con código 192 ya existe en la DB. Correspone al usuario HERNANDEZ FUENTES, RUBEN SEBASTIAN y no fue necesario insertarla nuevamente.
           Inserción de datos terminado.
           El usuario HERNANDEZ TAPIA, ANGEL CUSTODIO ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario HERNANDEZ TAPIA, NELLY DE LAS MERCEDES ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario HERRERA JARA, RUPERTO MARIO ya existe en la DB. No fue necesario insertarlo nuevamente.
           Inserción de datos terminado.
           El usuario HERRERA TORRES, MARINA DE LAS MERC

           El usuario VÁSQUEZ AMIGO, VANESSA ROMINA ya existe en la DB. No fue necesario insertarlo nuevamente.
           La remuneración con código 381 ya existe en la DB. Correspone al usuario VÁSQUEZ AMIGO, VANESSA ROMINA y no fue necesario insertarla nuevamente.
           Inserción de datos terminado.
           El usuario VÁSQUEZ GALVEZ, LUIS GABRIEL ya existe en la DB. No fue necesario insertarlo nuevamente.
           La remuneración con código 382 ya existe en la DB. Correspone al usuario VÁSQUEZ GALVEZ, LUIS GABRIEL y no fue necesario insertarla nuevamente.
           Inserción de datos terminado.
           El usuario ZUÑIGA LEIVA, RAUL DAVID ya existe en la DB. No fue necesario insertarlo nuevamente.
           La remuneración con código 383 ya existe en la DB. Correspone al usuario ZUÑIGA LEIVA, RAUL DAVID y no fue necesario insertarla nuevamente.
           Inserción de datos terminado.
           El usuario ZÚÑIGA HERRERA, PILAR ANDREA ya existe en la DB. No fue necesar

In [5]:
web_scraper.date_remunerations_dict['2019']['Enero']['Talca']

[     ANO    MES       ESTAMENTO                      NOMBRE COMPLETO  \
 0   2019  ENERO     PROFESIONAL         ABARCA VALDES, PAULINA ELENA   
 1   2019  ENERO  ADMINISTRATIVO        AGUILERA LOYOLA, MAC N ICOLAS   
 2   2019  ENERO     PROFESIONAL  AGURTO VILLAGRA, VICTORIA ELIZABETH   
 3   2019  ENERO     PROFESIONAL      ALCAINO REYES, PAULINA PATRICIA   
 4   2019  ENERO  ADMINISTRATIVO          ANDAUR ROJAS, MONICA ISABEL   
 ..   ...    ...             ...                                  ...   
 95  2019  ENERO  ADMINISTRATIVO   MORALES CASTILLO, CRISTIAN RODRIGO   
 96  2019  ENERO  ADMINISTRATIVO      MORALES ORTEGA, MARIA FRANCISCA   
 97  2019  ENERO  ADMINISTRATIVO          MORENO CALZADA, MARIO OSCAR   
 98  2019  ENERO     PROFESIONAL         MUÑOZ ANACONA, DAISY VANESSA   
 99  2019  ENERO  ADMINISTRATIVO           MUÑOZ JAQUE, MARIO ESTEBAN   
 
                    CARGO O FUNCION GRADO EUS / CARGO CON JORNADA  \
 0      ENCARGADA OFI. ADULTO MAYOR                  